In [1]:
# Environment Check: GPU Availability
import subprocess
import sys
result = subprocess.run(['bash', '-lc', 'nvidia-smi || true'], capture_output=True, text=True)
print(result.stdout)
if 'NVIDIA-SMI' not in result.stdout:
    print('GPU not available. Exiting competition.')
    # Note: In a real scenario, we would call exit(), but since we can't, we'll flag it.
else:
    print('GPU is available. Proceeding.')

Sun Sep 28 05:10:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import json
import pandas as pd
from collections import Counter
import os

# Load train metadata
with open('train_metadata.json', 'r') as f:
    train_meta = json.load(f)

print('Train metadata keys:', train_meta.keys())
print('Number of train samples:', len(train_meta['annotations']))

# Convert to DataFrame for easier exploration
train_df = pd.DataFrame(train_meta['annotations'])
print('\nTrain DataFrame shape:', train_df.shape)
print('\nTrain DataFrame columns:', train_df.columns.tolist())
print('\nUnique classes in train:', train_df['category_id'].nunique())
print('\nClass distribution (top 10):')
class_counts = Counter(train_df['category_id'])
print(sorted(class_counts.items(), key=lambda x: x[1], reverse=True)[:10])

# More class stats
print('\nImages per class - min:', min(class_counts.values()))
print('max:', max(class_counts.values()))
print('mean:', sum(class_counts.values()) / len(class_counts))

# Institution distribution
inst_counts = Counter(train_df['institution_id'])
print('\nNumber of institutions:', len(inst_counts))
print('Institution samples (top 5):', sorted(inst_counts.items(), key=lambda x: x[1], reverse=True)[:5])

# Check train images structure
print('\nTrain images dir contents:', os.listdir('train_images')[:5])  # Should show subdirs

# Load test metadata - it's a list, not dict
with open('test_metadata.json', 'r') as f:
    test_meta = json.load(f)

print('\nTest metadata type:', type(test_meta))
print('Number of test samples:', len(test_meta))

test_df = pd.DataFrame(test_meta)
print('\nTest DataFrame shape:', test_df.shape)
print('\nTest DataFrame columns:', test_df.columns.tolist())

# Check test images structure
print('\nTest images dir contents:', os.listdir('test_images')[:5])

Train metadata keys: dict_keys(['annotations', 'categories', 'distances', 'genera', 'images', 'institutions', 'license'])
Number of train samples: 665720



Train DataFrame shape: (665720, 4)

Train DataFrame columns: ['category_id', 'genus_id', 'image_id', 'institution_id']

Unique classes in train: 15501

Class distribution (top 10):
[(19, 64), (123, 64), (207, 64), (230, 64), (231, 64), (297, 64), (364, 64), (572, 64), (573, 64), (578, 64)]

Images per class - min: 4
max: 64
mean: 42.946906651183795

Number of institutions: 60
Institution samples (top 5): [(36, 247771), (52, 88888), (21, 65291), (38, 39389), (53, 28159)]

Train images dir contents: ['036', '110', '037', '107', '144']

Test metadata type: <class 'list'>
Number of test samples: 174052



Test DataFrame shape: (174052, 3)

Test DataFrame columns: ['file_name', 'image_id', 'license']

Test images dir contents: ['036', '110', '037', '107', '162']


In [5]:
# Install PyTorch with CUDA 12.1 support and other dependencies
import subprocess
import sys
import os
from pathlib import Path
import shutil

def pip_install(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Clean any prior torch installations
for pkg in ('torch', 'torchvision', 'torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Remove stray directories
stray_dirs = [
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch'
]
for d in stray_dirs:
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Install PyTorch cu121
pip_install('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1'
)

# Create constraints
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# Install other deps
pip_install('install', '-c', 'constraints.txt',
    'timm==1.0.3',
    'albumentations==1.4.8',
    'scikit-learn==1.5.0',
    'pandas==2.2.2',
    'numpy==1.26.4',
    'pillow==10.4.0',
    'tqdm==4.66.5',
    '--upgrade-strategy', 'only-if-needed'
)

# Verify installation
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
assert torch.cuda.is_available(), 'CUDA not available after install'

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 524.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 456.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 189.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 540.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 549.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 477.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 478.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 479.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 425.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 526.6 MB/s eta 0:00:00


PyTorch version: 2.4.1+cu121
CUDA available: True
CUDA version: 12.1
GPU: NVIDIA A10-24Q


# Updated Plan for Herbarium 2022 (Post-Expert Review)

## Dataset Overview
- Train: 665,720 images, 15,501 classes (long-tail: 4-64 images/class, mean ~43), 2,564 genera
- Test: 174,052 images
- Images loaded via file_name: e.g., 'train_images/028/14/02814__046.jpg' (verified paths exist)
- Metadata: train_df with category_idx (0-15500), genus_idx (0-2563), institution_id (groups for CV)
- Mappings saved: cat_mapping.pkl, genus_mapping.pkl for submission decoding

## Challenges & Key Insights
- Extreme long-tail: Use class-balanced sampling, not subsampling (full data essential for macro F1)
- Institution leakage: CV must group by institution_id (mean 7 inst/class, min 1)
- Hierarchical: Leverage genus_id for multi-task learning to boost rare species

## Updated Baseline Strategy
1. **Data Loading**: Custom PyTorch Dataset: load PIL.Image.open(f'{split}_images/{file_name}'), convert to RGB tensor, normalize per timm (mean/std ImageNet). Return img, category_idx, genus_idx.
2. **CV**: From sklearn.model_selection import StratifiedGroupKFold; sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42); folds = list(sgkf.split(train_df, train_df['category_idx'], groups=train_df['institution_id'])). Start with 2 folds for speed, monitor class coverage per fold.
3. **Sampler**: Class-Balanced WeightedRandomSampler (beta=0.9999) based on class_freq, or inverse sqrt(freq) weights to oversample tails while keeping batch diversity.
4. **Model**: timm.create_model('convnext_tiny.in22k_ft_in1k', pretrained=True, num_classes=15501, in_chans=3); Add auxiliary genus head: fc_genus = nn.Linear(model.head.fc.in_features, 2564); Dual loss: CE(species_logits, category_idx, label_smoothing=0.05) + 0.3 * CE(genus_logits, genus_idx). Use AMP for mixed precision.
5. **Augmentations**: Albumentations train: RandomResizedCrop(384, scale=(0.7,1.0)), HorizontalFlip, ShiftScaleRotate, ColorJitter, CoarseDropout; Val: Resize(384), CenterCrop(384). Add Mixup/CutMix (alpha=0.2) during training.
6. **Training**: Full dataset. DataLoader batch_size=32 (accumulate_gradients=8 for effective 256), num_workers=8, pin_memory=True. AdamW(lr=3e-4, weight_decay=0.05), cosine scheduler with 2-epoch warmup, 15 epochs, EMA (decay=0.999). Early stopping on val macro F1.
7. **Metrics**: Macro F1 (sklearn.metrics.f1_score(average='macro')) on val folds; track by rarity bins (e.g., classes <10 imgs).
8. **Inference**: TTA (4-8: flips + multi-crop), average logits over folds. Post-process: logit_adjust = logits - log(class_freq[pred_idx]); then softmax, argmax. Decode with idx2cat for submission.csv (predicted_category_id column).

## Next Steps
- Create '01_baseline_model.ipynb': Implement Dataset, Collate with Mixup, Model with dual heads, Sampler, train/eval loop for 1 fold to validate pipeline (target val macro F1 >0.50).
- If successful, train 5 folds, generate OOF and test preds, blend if needed.
- Improvements: Add ArcFace head for species (s=30, m=0.5), institution embedding, pseudo-labeling on high-conf test.
- Request expert review after 1-fold training results and code review.

In [6]:
# Build proper DataFrames with file_names and verify paths
import random

# For train: merge annotations with images to get file_name
train_images_df = pd.DataFrame(train_meta['images'])
train_df = train_df.merge(train_images_df[['image_id', 'file_name']], on='image_id', how='left')
print('Train DF after merge shape:', train_df.shape)
print('Columns:', train_df.columns.tolist())
print('Sample file_name:', train_df['file_name'].iloc[0])

# Verify paths for random samples
random.seed(42)
sample_ids = random.sample(train_df['image_id'].tolist(), 10)
for img_id in sample_ids:
    fname = train_df[train_df['image_id'] == img_id]['file_name'].iloc[0]
    path = os.path.join('train_images', fname)
    exists = os.path.exists(path)
    print(f'Image {img_id}: {fname} exists: {exists}')
    if not exists:
        print('PATH ISSUE!')

# For test: already has file_name
print('\nTest sample file_name:', test_df['file_name'].iloc[0])
test_sample_paths = random.sample(test_df['file_name'].tolist(), 5)
for fname in test_sample_paths:
    path = os.path.join('test_images', fname)
    exists = os.path.exists(path)
    print(f'Test {fname} exists: {exists}')

# Category mapping: remap to 0..N-1
unique_cats = sorted(train_df['category_id'].unique())
cat2idx = {cat: idx for idx, cat in enumerate(unique_cats)}
idx2cat = {idx: cat for cat, idx in cat2idx.items()}
train_df['category_idx'] = train_df['category_id'].map(cat2idx)
print('\nNumber of categories:', len(unique_cats))
print('Category range:', min(cat2idx.values()), 'to', max(cat2idx.values()))

# Genus mapping
unique_genus = sorted(train_df['genus_id'].unique())
genus2idx = {gn: idx for idx, gn in enumerate(unique_genus)}
train_df['genus_idx'] = train_df['genus_id'].map(genus2idx)
print('Number of genera:', len(unique_genus))

# Class frequencies for sampler/weights
class_freq = Counter(train_df['category_idx'])
print('\nClass freq min/max/mean:', min(class_freq.values()), max(class_freq.values()), sum(class_freq.values()) / len(class_freq))

# Save mappings
import pickle
with open('cat_mapping.pkl', 'wb') as f:
    pickle.dump({'cat2idx': cat2idx, 'idx2cat': idx2cat}, f)
with open('genus_mapping.pkl', 'wb') as f:
    pickle.dump({'genus2idx': genus2idx}, f)

# Institution groups for CV
print('\nInstitutions per class (sample):', train_df.groupby('category_id')['institution_id'].nunique().describe())

Train DF after merge shape: (665720, 5)
Columns: ['category_id', 'genus_id', 'image_id', 'institution_id', 'file_name']
Sample file_name: 000/00/00000__001.jpg
Image 02814__046: 028/14/02814__046.jpg exists: True
Image 00611__036: 006/11/00611__036.jpg exists: True
Image 06750__025: 067/50/06750__025.jpg exists: True
Image 06039__039: 060/39/06039__039.jpg exists: True
Image 05492__019: 054/92/05492__019.jpg exists: True
Image 03499__011: 034/99/03499__011.jpg exists: True


Image 02608__028: 026/08/02608__028.jpg exists: True
Image 13307__006: 133/07/13307__006.jpg exists: True
Image 02227__005: 022/27/02227__005.jpg exists: True
Image 14403__034: 144/03/14403__034.jpg exists: True

Test sample file_name: 000/test-000000.jpg
Test 110/test-110604.jpg exists: True
Test 008/test-008331.jpg exists: True
Test 007/test-007811.jpg exists: True
Test 024/test-024561.jpg exists: True
Test 057/test-057314.jpg exists: True

Number of categories: 15501
Category range: 0 to 15500
Number of genera: 2564

Class freq min/max/mean: 4 64 42.946906651183795

Institutions per class (sample): count    15501.000000
mean         7.146313
std          3.038974
min          1.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         21.000000
Name: institution_id, dtype: float64
